In [1]:
import pdb
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

In [2]:
path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: 
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])     

In [3]:
# symoblic differentiation, nice
from sympy import symbols,diff
y1 ,y = symbols('y1 y')
diff((y1 -y)**2, y1)

-2*y + 2*y1

# Foundations:

In [4]:
n, m = x_train.shape
c = y_train.max()+1 # number of classes in out
n,m,c

(50000, 784, tensor(10))

In [5]:
# num hidden size
nh = 50

In [6]:
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)

w2 = torch.randn(nh, 1) # for now use mse, out int from 0 to 9 to match on mse
b2 = torch.zeros(1)
# mse to simplify starting point

In [7]:
def lin(x, w, b):
    return x@w +b

In [8]:
t = lin(x_valid, w1,b1)
t.shape

torch.Size([10000, 50])

In [9]:
def relu(x): 
    return x.clamp_min(0.)

In [10]:
t = relu(t)
t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

# First MLP:

In [11]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2,w2,b2)

In [12]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

# Add loss function: MSE

In [13]:
res.shape, y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [14]:
(res-y_valid).shape # bad cuz of broadcasting rules!

torch.Size([10000, 10000])

In [15]:
res[:, 0].shape # first el of all rows (which is just the only el in this case)

torch.Size([10000])

In [16]:
# we could use squeeze() to remove ALL UNIT DIMENSIONS
res[:, 0]; res[:, 0].shape

torch.Size([10000])

In [17]:
((res[:, 0] - y_valid)**2).mean()

tensor(4154.01)

In [18]:
def mse(yhat, y):
    # it is better to kee the [:, 0] inside the function to avoid any preprocessing
    # outside the func itself
    return ((yhat[:, 0] - y)**2).mean()

In [19]:
mse(res,y_valid)

tensor(4154.01)

# Backward pass

Chain rule:
suppose you have: 
    $$y=3u+9, $$ where $$ u=x^{2}$$
then:
$$ \frac{dy}{dx}= \frac{dy}{du} * \frac{du}{dx} = 3 * 2x = 6x $$
ChainRule baby!

In [20]:
def model(xb):
    # 4 steps l1 relu l2 mse
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2,w2,b2)

def lin(inp, w, b):
    return inp@w +b

def relu(x): 
    return x.clamp_min(0.)

In [21]:
def lin_grad(inp, out, w, b):
    # grad of matmul wrt input
    inp.g = out.g @ w.t() 
    
    w.g = inp.t() @ out.g
    b.g = out.g.sum(0) 
    # sum collapsing 0 dim since it has been broadcasted on/across cols
    
def forward_and_backward(inp, targ):

    # forward pass    
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)

    diff = out[:,0]-targ
    loss = diff.pow(2).mean()
    
    # backward pass
    # each .g accumulates all prev grads products
    out.g = 2.* (diff[:,None] / inp.shape[0])    

    lin_grad(inp=l2, out=out, w=w2, b=b2)
    l1.g = (l1>0).float() * l2.g # relu filters gradient flow to previous layers
    lin_grad(inp=inp, out=l1, w=w1, b=b1)

In [22]:
forward_and_backward(x_train,y_train)

In [23]:
# shapes: 
# x = (50k, 784)
# w1 = (784, 50)
# b1 = (50) # this col vect gets broadcasted to each col of x@w1
# l1 = x@w+b =(50k, 50) # each entry of this matrix is dot prod of
# # an (img, col of weights) + bias 
# 
# l2 = relu(l1) # drops all entries below 0
# l2 = (50k, 50)
# 
# w2 = (50, 1) 
# b2 = (1)
# 
# out = l2 @ w2 +b2 # b2 gets broadcasted to match out dims of matmul
# out = (50k,1) + (50k,1)
# out = (50k,1)
# 
# out - T = (50k,1)
# 
# then we have the loss.
# the derivative of the mse is
# 
# dL/dY = 2(Y-T)/N <- grad of output, shape (50k, 1)
# 
# Y = l2 @ w2 + b2 
# so Y depends on 3 vals: l2, w2, b2 
# so we must take derivatives wrt all 3 vals
# dY/dl2 = w2 <- the grad of l2
# dY/w2 = l2 <- the grad of w2
# dY/b2 = 1 <- the grad of b2
# 
# but we want the grads for l2, w2, b2 wrt how they impact on the Loss
# thus for to get the grad-impacts of l2,w2,b2 on the loss we have to 
# multiply each one of [dY/dl2, dY/w2, dY/b2] by dL/dY <- this is the grad of
# the output, it has already been computed in backprop -> we need to take out
# as input aswell to multiply it for the reason just described.
# 
# each col of w_i is a set of weights used to model/map all input obs into another
# dimensional space + bias/eps

# Testing

In [24]:
# let's save the gradients s.t. test our code
def get_grad(x):
    return x.g.clone()

# this is a cool way to define els inside list and list at once
chks = w1,w2,b1,b2, x_train
grads = w1_g, w2_g,b1_g,b2_g, inp_g = map(get_grad, chks)


def mkgrad(x):
    return x.clone().requires_grad_(True)

ptgrads = w1_copy, w2_copy, b1_copy, b2_copy, xt_copy = map(mkgrad, chks)

def forward(inp, targets):
    l1 = lin(inp, w1_copy, b1_copy)
    l2 = relu(l1)
    out = lin(l2, w2_copy, b2_copy)
    return mse(out, targets)

loss = forward(xt_copy, y_train)
loss.backward()

for a,b in zip(grads, ptgrads):
    test-close(a.grad, b, eps=0.01)

# Refactoring

In [25]:
# recalling that by modifiying an object that is stored in other classes implies that classes contnent is changed

class A():
    def __init__(self, data): self.x = data
        
class B():
    def __init__(self, data): self.x = data
        
class  C():
    def __init__(self, data): self.x = data

c = C(8) 
a = A(c)
b = B(c)
print(a.x.x, b.x.x)
c.x = 9
a.x.x, b.x.x

8 8


(9, 9)

In [26]:
class Relu():
    def __call__(self, inp):
        # it stores its own input, output
        self.inp = inp
        self.out = inp.clamp_min(0.) 
        return self.out
    
    def backward(self):
        # self.imp.g = relu's derivative  * chainRule: 
            # all the grad up untill now in chain
        self.inp.g = (self.inp>0).float() * self.out.g

In [27]:
class LinearLayer():
    def __init__(self, w, b):
        self.w, self.b = w, b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = lin(self.inp, self.w, self.b)
        return self.out 
    
    def backward(self):
        # when calling backward for the first time
        # self.out.g has been already modified: contains the grad computed by Mse.backward()
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(dim=0)

In [28]:
class Mse():
    def __call__(self, inp, target):
        # inp is the out of prev layer
        self.inp, self.target = inp, target
        self.out = mse(self.inp, self.target)
        return self.out
    
    def backward(self):
        self.inp.g = 2*(self.inp.squeeze() - self.target).unsqueeze(-1) / self.target.shape[0]

In [29]:
class Model():
    def __init__(self, w1, w2, b1, b2):
        self.layers = [LinearLayer(w1, b1), Relu(), LinearLayer(w2, b2)] 
        self.loss = Mse()
        
    def __call__(self, x, target):
        for l in self.layers:
            x = l(x)
        return self.loss(x, target)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): # from last layer to first layer!
            l.backward()

In [30]:
model = Model(w1, w2, b1, b2)
model(x_train, y_train)
model.backward()

In [31]:
test_close(w1_g, w1.g, eps=0.01)
test_close(w2_g, w2.g, eps=0.01)
test_close(b1_g, b1.g, eps=0.01)
test_close(b2_g, b2.g, eps=0.01)
test_close(inp_g, x_train.g, eps=0.01)

# Further Refactoring

As we can see above, each class that we created above, in their __call__ method there is repeated calls: self.in = in; compute out and return self.out...

All backward calls need self.out

In [32]:
class Module():
    def __call__(self, *args):
        self.args = args # store all things that are passed in a list
        self.out = self.forward(*args) # calls derived forward()
        return self.out
    
    def forward(self): 
        raise Exception("Not implemented")
        
    def backward(self):
        # All backward calls need self.out to apply
        # chainRule: all the grad up untill now in chain
        # this takes *self.args that is particular for each derived forward() signature 
        self.bwd(self.out, *self.args) # list unpacking

    def bwd(self):
        raise Exception("Not implemented")

In [33]:
class Relu(Module): # inherit Module "interface" (nn.Module in pytorch)

    # now coder is required to implemnt only 
    # forward and how to get derivative of this forward * gradient up untill now (out.g); much cleaner
    
    def forward(self, inp): 
        '''
        example usage:
        relu = Relu()
        x1 = relu(x)
        '''
        return inp.clamp_min(0.)
    
    def bwd(self, out, inp):
        inp.g = (inp>0).float() * out.g

In [34]:
class LinearLayer(Module):
    def __init__(self, w, b):
        self.w, self.b = w, b 
        
    def forward(self, inp):
        return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(dim=0)

In [35]:
class Mse(Module):
    def forward(self, inp, targets):
        self.diff = inp.squeeze() - targets
        return self.diff.pow(2).mean()
        
    def bwd(self, _, inp, targets): # _ = out
        inp.g = 2. * self.diff.unsqueeze(-1) /targets.shape[0]

In [36]:
model = Model(w1, w2, b1, b2)
model(x_train, y_train)
model.backward()

In [37]:
test_close(w1_g, w1.g, eps=0.01)
test_close(w2_g, w2.g, eps=0.01)
test_close(b1_g, b1.g, eps=0.01)
test_close(b2_g, b2.g, eps=0.01)
test_close(inp_g, x_train.g, eps=0.01)

# Autograd
So now we have built on our own derivatives computation for layers. 
So we can now use autograd

In [38]:
from torch import nn
import torch.nn.functional as F

In [49]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in, n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    
    def forward(self, inp):
        return inp@self.w + self.b

In [55]:
class Model(nn.Module):
    
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in, nh), nn.ReLU(), Linear(nh, n_out)]
        
    def __call__(self, x, targets):
        for l in self.layers:
            x = l(x)
        return F.mse_loss(x, targets[:,None].float())

In [56]:
model = Model(784, 50, 1)
loss = model(x_train, y_train)
loss.backward()